## A. Configuración General.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#3. Constantes.
%run "../constantes.ipynb"

dataset_con_fe = dataset_file_fe_12_3
cantidad_meses_train = 12
ventana = 3
mes_train = mes_train_ult_anio
mes_test = mes_test

In [ ]:
#3. Funciones
%run "../funciones.ipynb"

In [ ]:
#4. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)

In [ ]:
#iv. Divido entre Train y Test.
#a. Train.
X = data[data['foto_mes'].isin(mes_train)]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

#b. Test.
X_futuro = data[data['foto_mes'] == mes_test]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

## B. Optimización Hiperparámetros (OH) con cantidad_meses_train meses con df -ventana con ratios incluidos.

In [ ]:
#a. Funcion de optimización de hiperparámetros.
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 2000)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
    max_features = trial.suggest_float('max_features', 0.05, 0.7)

    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

    model.fit(X, y)

    return ganancia_prob(model.oob_decision_function_, y)

In [ ]:
#2. Voy a realizar un estudio de Optuna para encontrar los mejores parámetros.
#i. Creo la base de datos donde guardar los resultados.
storage_name = "sqlite:///" + db_path + "optimization_tree.db"
study_name = f"exp_rf_{cantidad_meses_train}_{ventana}" # Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting.

#ii. Creo el estudio.
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

#iii. Corro el estudio.
study.optimize(objective, n_trials=100)

## C. Entrenamiento con mejores Hiperparámetros.

In [ ]:
#1. Tomamos el mejor modelo.
model_rf = RandomForestClassifier(
        n_estimators=100,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

In [ ]:
#2. Entrenamos.
model_rf.fit(X, y)

## D. Guardamos el modelo entrenado.

In [ ]:
#1. Guardamos el modelo recién generado.
filename = modelos_path + 'rf__{}_{}.sav'.format(cantidad_meses_train,ventana)
pickle.dump(model_rf, open(filename, 'wb'))

In [ ]:
model_rf = pickle.load(open(filename, 'rb'))
model_rf